In [ ]:
lab=True
epochs=30

In [ ]:
#apt-get install git python3-opencv
import os

if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/joaopauloschuler/k-neural-api
   3bf7a52..5dcd9ea  master     -> origin/master
Updating 3bf7a52..5dcd9ea
Fast-forward
 cai/models.py | 83 +++++++++++++++++++++++++++++++++++++++++++++--------------
 1 file changed, 64 insertions(+), 19 deletions(-)
Processing /root/k
  Created wheel for cai: filename=cai-0.0.7-py3-none-any.whl size=15976 sha256=14b9b8a59ea5ae45a12edd47fb53ced37bfb072c4200bcbfb85890b87af86eaf
  Stored in directory: /tmp/pip-ephem-wheel-cache-9otbse22/wheels/6e/59/4f/44088eb1d402e7b2bebb24aef094469d18d276d3db6cc13c0d
Successfully built cai
  Attempting uninstall: cai
    Found existing installation: cai 0.0.7
    Uninstalling cai-0.0.7:
      Successfully uninstalled cai-0.0.7


In [ ]:
import cai.layers
import cai.datasets
import cai.models
import numpy as np
from keras import backend
from keras import layers
import keras.applications
import keras.applications.inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras_applications import imagenet_utils
from keras_applications import get_submodules_from_kwargs
from keras_applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import np_utils
from sklearn.utils import class_weight
from skimage import color as skimage_color
import gc
from keras.callbacks import EarlyStopping
import multiprocessing
import random
import glob

Using TensorFlow backend.


In [ ]:
def load_plantvillage(seed=None, root_dir=None, lab=False, 
  verbose=True, bipolar=False, base_model_name='plant_leaf'):
  if root_dir == None:
    print("define where the plantvillage folder is.")
    return
  random.seed(seed)
  def read_from_paths(paths):
    x=[]
    for path in paths:
      img = load_img(path, target_size=(224,224))
      img = img_to_array(img)
      x.append(img)
    return x
  
  classes = os.listdir(root_dir)
  classes = sorted(classes)

  train_path = []
  val_path = []
  test_path = []

  train_x,train_y = [],[]
  val_x,val_y = [],[]
  test_x,test_y =[],[]

  #read path and categorize to three groups , 6,2,2
  for i,_class in enumerate(classes):
      paths = glob.glob(os.path.join(root_dir,_class,"*"))
      paths = [n for n in paths if n.endswith(".JPG") or n.endswith(".jpg")]
      random.shuffle(paths)
      cat_total = len(paths)
      train_path.extend(paths[:int(cat_total*0.6)])
      train_y.extend([i]*int(cat_total*0.6))
      val_path.extend(paths[int(cat_total*0.6):int(cat_total*0.8)])
      val_y.extend([i]*len(paths[int(cat_total*0.6):int(cat_total*0.8)]))
      test_path.extend(paths[int(cat_total*0.8):])
      test_y.extend([i]*len(paths[int(cat_total*0.8):]))
  
  if (verbose):
    print ("loading train images")
  train_x = np.array(read_from_paths(train_path), dtype='float16')
  
  if (verbose):
    print ("loading validation images")
  val_x = np.array(read_from_paths(val_path), dtype='float16')

  if (verbose):
    print ("loading test images")
  test_x = np.array(read_from_paths(test_path), dtype='float16')

  train_y = np.array(train_y)
  val_y = np.array(val_y)
  test_y = np.array(test_y)

  #convert everything to numpy
  #train_x = np.array(train_x)/255.
  #val_x = np.array(val_x)/255.
  #test_x = np.array(test_x)/255.

  if (lab):
        # LAB datasets are cached
        cachefilename = 'cache-lab-'+base_model_name+'-'+str(bipolar)+'-'+str(train_x.shape[1])+'-'+str(train_x.shape[2])+'.npz'
        if (verbose):
            print("Converting RGB to LAB: "+cachefilename)
        if not os.path.isfile(cachefilename):            
            train_x /= 255
            val_x /= 255
            test_x /= 255
            if (verbose):
                print("Converting training.")
            cai.datasets.skimage_rgb2lab_a(train_x,  verbose)
            if (verbose):
                print("Converting validation.")
            cai.datasets.skimage_rgb2lab_a(val_x,  verbose)
            if (verbose):
                print("Converting test.")
            cai.datasets.skimage_rgb2lab_a(test_x,  verbose)
            gc.collect()
            if (bipolar):
                # JP prefers bipolar input [-2,+2]
                train_x[:,:,:,0:3] /= [25, 50, 50]
                train_x[:,:,:,0] -= 2
                val_x[:,:,:,0:3] /= [25, 50, 50]
                val_x[:,:,:,0] -= 2
                test_x[:,:,:,0:3] /= [25, 50, 50]
                test_x[:,:,:,0] -= 2
            else:
                train_x[:,:,:,0:3] /= [100, 200, 200]
                train_x[:,:,:,1:3] += 0.5
                val_x[:,:,:,0:3] /= [100, 200, 200]
                val_x[:,:,:,1:3] += 0.5
                test_x[:,:,:,0:3] /= [100, 200, 200]
                test_x[:,:,:,1:3] += 0.5
            #if (verbose):
              #print("Saving: "+cachefilename)
              #np.savez_compressed(cachefilename, a=train_x,  b=val_x, c=test_x)
        else:
            if (verbose):
              print("Loading: "+cachefilename)
            loaded = np.load(cachefilename)
            train_x = loaded['a']
            val_x = loaded['b']
            test_x = loaded['c']
            gc.collect()
  else:
        if (verbose):
            print("Loading RGB.")
        if (bipolar):
            train_x /= 64
            val_x /= 64
            test_x /= 64
            train_x -= 2
            val_x -= 2
            test_x -= 2
        else:
            train_x /= 255
            val_x /= 255
            test_x /= 255

  if (verbose):
        for channel in range(0, train_x.shape[3]):
            sub_matrix = train_x[:,:,:,channel]
            print('Channel ', channel, ' min:', np.min(sub_matrix), ' max:', np.max(sub_matrix))
  #calculate class weight
  classweight = class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)

  #convert to categorical
  train_y = np_utils.to_categorical(train_y, 38)
  val_y = np_utils.to_categorical(val_y, 38)
  test_y = np_utils.to_categorical(test_y, 38)
  print("loaded")

  return train_x,val_x,test_x,train_y,val_y,test_y,classweight,classes

In [ ]:
url_zip_file="https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
Verbose=True
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file, 
    expected_folder_name=expected_folder_name, Verbose=Verbose)

In [ ]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))

rm: cannot remove 'plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves': No such file or directory
['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Se

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = load_plantvillage(seed=7, root_dir=data_dir, lab=lab)
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

loading train images
loading validation images
loading test images
Converting RGB to LAB: cache-lab-plant_leaf-False-224-224.npz
Converting training.
0  images converted to lab.
1000  images converted to lab.
2000  images converted to lab.
3000  images converted to lab.
4000  images converted to lab.
5000  images converted to lab.
6000  images converted to lab.
7000  images converted to lab.
8000  images converted to lab.
9000  images converted to lab.
10000  images converted to lab.
11000  images converted to lab.
12000  images converted to lab.
13000  images converted to lab.
14000  images converted to lab.
15000  images converted to lab.
16000  images converted to lab.
17000  images converted to lab.
18000  images converted to lab.
19000  images converted to lab.
20000  images converted to lab.
21000  images converted to lab.
22000  images converted to lab.
23000  images converted to lab.
24000  images converted to lab.
25000  images converted to lab.
26000  images converted to lab.

In [ ]:
for two_paths_second_block in [False, True]:
  for l_ratio in [0.1, 0.2, 0.33, 0.5, 0.66, 0.8, 0.9]:
    basefilename = 'two-path-inception-v6-'+str(two_paths_second_block)+'-'+str(l_ratio)
    print('Running: '+basefilename)
    model = cai.models.compiled_two_path_inception_v3(
      input_shape=(224,224,3),
      classes=38, 
      two_paths_first_block=True,
      two_paths_second_block=two_paths_second_block,
      l_ratio=l_ratio,
      ab_ratio=(1-l_ratio),
      max_mix_idx=5, 
      model_name='two_path_inception_v3'
      )    
    monitor='val_accuracy'
    best_result_file_name = basefilename+'-best_result.hdf5'
    save_best = keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name, 
      monitor=monitor, 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False, 
      mode='max', 
      period=1)
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=32,
      validation_data=(val_x,val_y),
      callbacks=[save_best],class_weight=classweight,
      workers=multiprocessing.cpu_count())
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    cai.models.save_model(model, basefilename)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)

Running: two-path-inception-v6-False-0.1
Train on 32570 samples, validate on 10857 samples
Epoch 1/30
32570/32570 [==============================] - 248s 8ms/step - loss: 1.1781 - accuracy: 0.6951 - top_k_categorical_accuracy: 0.9084 - val_loss: 1.3065 - val_accuracy: 0.6274 - val_top_k_categorical_accuracy: 0.8733

Epoch 00001: val_accuracy improved from -inf to 0.62743, saving model to two-path-inception-v6-False-0.1-best_result.hdf5
Epoch 2/30
32570/32570 [==============================] - 244s 7ms/step - loss: 0.4065 - accuracy: 0.8909 - top_k_categorical_accuracy: 0.9918 - val_loss: 0.6714 - val_accuracy: 0.7947 - val_top_k_categorical_accuracy: 0.9707

Epoch 00002: val_accuracy improved from 0.62743 to 0.79469, saving model to two-path-inception-v6-False-0.1-best_result.hdf5
Epoch 3/30
32570/32570 [==============================] - 245s 8ms/step - loss: 0.2334 - accuracy: 0.9359 - top_k_categorical_accuracy: 0.9973 - val_loss: 0.3562 - val_accuracy: 0.8921 - val_top_k_categorical

32570/32570 [==============================] - 244s 7ms/step - loss: 0.0081 - accuracy: 0.9988 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0230 - val_accuracy: 0.9929 - val_top_k_categorical_accuracy: 1.0000

Epoch 00027: val_accuracy did not improve from 0.99291
Epoch 28/30
32570/32570 [==============================] - 243s 7ms/step - loss: 0.0102 - accuracy: 0.9981 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0352 - val_accuracy: 0.9896 - val_top_k_categorical_accuracy: 0.9999

Epoch 00028: val_accuracy did not improve from 0.99291
Epoch 29/30
32570/32570 [==============================] - 242s 7ms/step - loss: 0.0077 - accuracy: 0.9990 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0285 - val_accuracy: 0.9905 - val_top_k_categorical_accuracy: 0.9998

Epoch 00029: val_accuracy did not improve from 0.99291
Epoch 30/30
32570/32570 [==============================] - 243s 7ms/step - loss: 0.0071 - accuracy: 0.9989 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0265 


Epoch 00022: val_accuracy improved from 0.99070 to 0.99079, saving model to two-path-inception-v6-False-0.2-best_result.hdf5
Epoch 23/30
32570/32570 [==============================] - 242s 7ms/step - loss: 0.0133 - accuracy: 0.9971 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0971 - val_accuracy: 0.9691 - val_top_k_categorical_accuracy: 0.9997

Epoch 00023: val_accuracy did not improve from 0.99079
Epoch 24/30
32570/32570 [==============================] - 242s 7ms/step - loss: 0.0122 - accuracy: 0.9976 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0349 - val_accuracy: 0.9893 - val_top_k_categorical_accuracy: 0.9998

Epoch 00024: val_accuracy did not improve from 0.99079
Epoch 25/30
32570/32570 [==============================] - 241s 7ms/step - loss: 0.0106 - accuracy: 0.9980 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0614 - val_accuracy: 0.9801 - val_top_k_categorical_accuracy: 0.9997

Epoch 00025: val_accuracy did not improve from 0.99079
Epoch 26/30
32570/32570 [

32570/32570 [==============================] - 238s 7ms/step - loss: 0.0218 - accuracy: 0.9954 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0562 - val_accuracy: 0.9827 - val_top_k_categorical_accuracy: 0.9998

Epoch 00017: val_accuracy improved from 0.98038 to 0.98268, saving model to two-path-inception-v6-False-0.33-best_result.hdf5
Epoch 18/30
32570/32570 [==============================] - 242s 7ms/step - loss: 0.0187 - accuracy: 0.9959 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0482 - val_accuracy: 0.9854 - val_top_k_categorical_accuracy: 0.9996

Epoch 00018: val_accuracy improved from 0.98268 to 0.98545, saving model to two-path-inception-v6-False-0.33-best_result.hdf5
Epoch 19/30
32570/32570 [==============================] - 238s 7ms/step - loss: 0.0169 - accuracy: 0.9967 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0671 - val_accuracy: 0.9782 - val_top_k_categorical_accuracy: 0.9997

Epoch 00019: val_accuracy did not improve from 0.98545
Epoch 20/30
32570/325

32570/32570 [==============================] - 239s 7ms/step - loss: 0.0439 - accuracy: 0.9886 - top_k_categorical_accuracy: 0.9999 - val_loss: 0.1782 - val_accuracy: 0.9444 - val_top_k_categorical_accuracy: 0.9979

Epoch 00011: val_accuracy did not improve from 0.94851
Epoch 12/30
32570/32570 [==============================] - 239s 7ms/step - loss: 0.0378 - accuracy: 0.9910 - top_k_categorical_accuracy: 0.9999 - val_loss: 0.0720 - val_accuracy: 0.9788 - val_top_k_categorical_accuracy: 0.9997

Epoch 00012: val_accuracy improved from 0.94851 to 0.97882, saving model to two-path-inception-v6-False-0.5-best_result.hdf5
Epoch 13/30
32570/32570 [==============================] - 239s 7ms/step - loss: 0.0318 - accuracy: 0.9931 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1201 - val_accuracy: 0.9619 - val_top_k_categorical_accuracy: 0.9997

Epoch 00013: val_accuracy did not improve from 0.97882
Epoch 14/30
32570/32570 [==============================] - 239s 7ms/step - loss: 0.0265 - acc

Epoch 6/30
32570/32570 [==============================] - 241s 7ms/step - loss: 0.1047 - accuracy: 0.9714 - top_k_categorical_accuracy: 0.9992 - val_loss: 0.1786 - val_accuracy: 0.9417 - val_top_k_categorical_accuracy: 0.9977

Epoch 00006: val_accuracy did not improve from 0.95035
Epoch 7/30
32570/32570 [==============================] - 244s 8ms/step - loss: 0.0825 - accuracy: 0.9785 - top_k_categorical_accuracy: 0.9997 - val_loss: 0.1512 - val_accuracy: 0.9516 - val_top_k_categorical_accuracy: 0.9988

Epoch 00007: val_accuracy improved from 0.95035 to 0.95155, saving model to two-path-inception-v6-False-0.66-best_result.hdf5
Epoch 8/30
32570/32570 [==============================] - 243s 7ms/step - loss: 0.0711 - accuracy: 0.9813 - top_k_categorical_accuracy: 0.9998 - val_loss: 0.1157 - val_accuracy: 0.9643 - val_top_k_categorical_accuracy: 0.9991

Epoch 00008: val_accuracy improved from 0.95155 to 0.96426, saving model to two-path-inception-v6-False-0.66-best_result.hdf5
Epoch 9/30
3

32570/32570 [==============================] - 248s 8ms/step - loss: 1.3190 - accuracy: 0.6561 - top_k_categorical_accuracy: 0.8793 - val_loss: 1.4922 - val_accuracy: 0.5648 - val_top_k_categorical_accuracy: 0.8707

Epoch 00001: val_accuracy improved from -inf to 0.56480, saving model to two-path-inception-v6-False-0.8-best_result.hdf5
Epoch 2/30
32570/32570 [==============================] - 237s 7ms/step - loss: 0.4866 - accuracy: 0.8656 - top_k_categorical_accuracy: 0.9881 - val_loss: 1.4358 - val_accuracy: 0.5743 - val_top_k_categorical_accuracy: 0.9084

Epoch 00002: val_accuracy improved from 0.56480 to 0.57428, saving model to two-path-inception-v6-False-0.8-best_result.hdf5
Epoch 3/30
32570/32570 [==============================] - 241s 7ms/step - loss: 0.2834 - accuracy: 0.9193 - top_k_categorical_accuracy: 0.9957 - val_loss: 0.4970 - val_accuracy: 0.8458 - val_top_k_categorical_accuracy: 0.9870

Epoch 00003: val_accuracy improved from 0.57428 to 0.84581, saving model to two-pat


Epoch 00026: val_accuracy did not improve from 0.98361
Epoch 27/30
32570/32570 [==============================] - 237s 7ms/step - loss: 0.0110 - accuracy: 0.9979 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0639 - val_accuracy: 0.9797 - val_top_k_categorical_accuracy: 0.9997

Epoch 00027: val_accuracy did not improve from 0.98361
Epoch 28/30
32570/32570 [==============================] - 238s 7ms/step - loss: 0.0086 - accuracy: 0.9989 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0454 - val_accuracy: 0.9861 - val_top_k_categorical_accuracy: 0.9997

Epoch 00028: val_accuracy improved from 0.98361 to 0.98609, saving model to two-path-inception-v6-False-0.8-best_result.hdf5
Epoch 29/30
32570/32570 [==============================] - 242s 7ms/step - loss: 0.0086 - accuracy: 0.9986 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1372 - val_accuracy: 0.9557 - val_top_k_categorical_accuracy: 0.9990

Epoch 00029: val_accuracy did not improve from 0.98609
Epoch 30/30
32570/32570 [


Epoch 00021: val_accuracy did not improve from 0.97964
Epoch 22/30
32570/32570 [==============================] - 240s 7ms/step - loss: 0.0187 - accuracy: 0.9963 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1004 - val_accuracy: 0.9677 - val_top_k_categorical_accuracy: 0.9992

Epoch 00022: val_accuracy did not improve from 0.97964
Epoch 23/30
32570/32570 [==============================] - 240s 7ms/step - loss: 0.0159 - accuracy: 0.9967 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0671 - val_accuracy: 0.9794 - val_top_k_categorical_accuracy: 0.9999

Epoch 00023: val_accuracy did not improve from 0.97964
Epoch 24/30
32570/32570 [==============================] - 240s 7ms/step - loss: 0.0141 - accuracy: 0.9970 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1710 - val_accuracy: 0.9455 - val_top_k_categorical_accuracy: 0.9982

Epoch 00024: val_accuracy did not improve from 0.97964
Epoch 25/30
32570/32570 [==============================] - 240s 7ms/step - loss: 0.0151 - accur

UnboundLocalError: local variable 'x' referenced before assignment